In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.runnables import RunnableBranch, RunnableLambda
from pydantic import BaseModel, Field
from typing import Literal


In [ ]:
load_dotenv()


model=ChatGoogleGenerativeAI(model="gemini-2.5-flash")

#parser takes the output and formats it in a structured manner
parser=StrOutputParser()


#basemodel acts as a secondary brain for a class providing a structure automatically at the very start
class Feedback(BaseModel):
    sentiment:Literal["positive","negative"]=Field(description="The sentiment of the customer")
parser2=PydanticOutputParser(pydantic_object=Feedback)


prompt1=PromptTemplate(template="""Give and classify the given feedback into either a positive or a negative sentiment {feedback}. Thereby, also give the output in the format: {response_format} """
    input_variables=["feedback"]
    partial_variables={"response_format":parser2.get_format_instructions()}
)


classifier_chain=prompt1|model|parser2

prompt2=PromptTemplate(template="""Give a detailed positive paragraph to the given feedback {feedback}.""",
    input_variables=["feedback"]
)



prompt3=PromptTemplate(template="""Give a detailed negative paragraph to the given feedback {feedback}.""",
    input_variables=["feedback"]
)


branch_chain=RunnableBranch(
    (lambda x: x.sentiment == "positive", prompt2 | model | parser),
    (lambda x: x.sentiment == "negative",prompt3 |model| parser),
    RunnableLambda(lambda x: "No valid  Response found")
)

chain=classifier_chain|branch_chain


result=chain.invoke({"feedback":"The product is good but the delivery was late"})

print(result)




















SyntaxError: invalid syntax. Perhaps you forgot a comma? (3115841276.py, line 13)